## FASE 3 : Análisis exploratorio de la rotación

En un entorno empresarial altamente competitivo, la retención de talento es un factor clave para el éxito organizacional.  
Este proyecto tiene como objetivo identificar los principales factores que influyen en la rotación de empleados (*Attrition*) y en la satisfacción laboral.

A través de un análisis exploratorio y comparativo, se evaluarán variables relacionadas con:

- Puesto de trabajo  
- Distancia al empleo  
- Promociones  
- Satisfacción laboral  
- Conciliación vida-trabajo  
- Diferencias por género  

El objetivo final es proporcionar insights estratégicos que ayuden a reducir la rotación y mejorar la experiencia laboral.
En este notebook se realiza el análisis de los datos con el objetivo de identificar patrones y relaciones clave asociados a la rotación de empleados dentro de la empresa.

In [1]:
# =========================
# IMPORTACIÓN DE LIBRERÍAS
# =========================
import numpy as np
import pandas as pd


# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [2]:
df_hr_clean = pd.read_csv("df_hr_clean.csv", index_col = 0)

In [3]:
df_hr_clean.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeNumber',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager', 'IncomeBand',
       'AgeGroup', 'TenureGroup', 'RiskScore'],
      dtype='object')

In [4]:
df_hr_clean['EmployeeNumber'].value_counts().sum() # Este dato nos servirá llegado el momento de la creación de la BBDD.

np.int64(1470)

In [5]:
df_hr_clean.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,IncomeBand,AgeGroup,TenureGroup,RiskScore
0,41,1,Rarely,1102,Sales,1,2,Life Sciences,1,2,Female,94,3,2,Sales Executive,4,Single,5993.0,19479.0,8,1,11,3,1,0,8,0,1,6,4,0,5,Medium,25-45,6-9,7
1,49,0,Frequently,279,Research & Development,8,1,Life Sciences,2,3,Male,61,2,2,Research Scientist,2,Married,5130.0,24907.0,1,0,23,4,4,1,10,3,3,10,7,1,7,Medium,Over 45,10+,2
2,37,1,Rarely,1373,Research & Development,2,2,Other,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090.0,2396.0,6,1,15,3,2,0,7,3,3,0,0,0,0,Medium,25-45,0-2,5
3,33,0,Frequently,1392,Research & Development,3,4,Life Sciences,5,4,Female,56,3,1,Research Scientist,3,Married,2909.0,23159.0,1,1,11,3,3,0,8,3,3,8,7,3,0,Medium,25-45,6-9,2
4,27,0,Rarely,591,Research & Development,2,1,Medical,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468.0,16632.0,9,0,12,3,4,1,6,3,3,2,2,2,2,Medium,25-45,0-2,3


0️⃣ Check rápido post-limpieza: realizamos una copia de nuestro Dataframe limpio y comprobamos que, efectivamente, no hay nulos en nuestro dataset.

In [3]:
# Vamos a trabajar sobre nuestra copia.
df = df_hr_clean.copy()
df.shape
df.isna().sum().sort_values(ascending=False)


Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
IncomeBand                  0
AgeGroup  

## 1️⃣ Entender la variable objetivo: Attrition (rotación)

Se analiza la proporción general de empleados que abandonan la empresa frente a aquellos que permanecen. Dado que Attrition es una variable binaria, su media representa directamente la tasa de rotación, lo que permite comparar el abandono entre distintos grupos de empleados.

- Attrition = 1 → empleados que abandonan  
- Attrition = 0 → empleados que permanecen 


In [7]:
# Número de empleados en la empresa.
df.shape     # Donde el número de filas devuelto, será el número de empleados.

(1470, 36)

In [8]:
# 🎯 Pregunta: ¿qué porcentaje de empleados abandona la empresa?
attrition_rate = (df['Attrition'].value_counts(normalize = True) *100).reset_index()
display(attrition_rate.round(2))

,Attrition,proportion
0,0,83.88
1,1,16.12


#### 💡Insight: 
Aunque la mayoría de empleados permanece en la empresa, existe un porcentaje significativo de rotación que puede representar un riesgo en términos de costes, productividad y clima laboral.

### Work-Life Balance, género y Attrition

Se explora cómo influye la conciliación vida-trabajo en la rotación, diferenciando por género.

El objetivo es detectar posibles desigualdades o colectivos más vulnerables.

In [9]:
# 🎯 Pregunta: ¿el género es significativo en el porcentaje de rotación?
attrition_gender = (df.groupby("Gender")["Attrition"].mean()* 100).reset_index()
display(attrition_gender.round(2))

,Gender,Attrition
0,Female,14.80
1,Male,17.01


In [10]:
# De las mujeres presentes en la empresa, el 14.80 % abandona su puesto. En cuanto a hombres, del total de presentes en la empresa,
# el 17.01% la abandona.

In [11]:
# Creamos un filtro cuya condición es que haya abandono de la empresa (attrition = 1)
mask_attrition = df['Attrition'] == 1

In [12]:
# Aplicamos el filtro para conocer el porcentaje de rotación por género
(df.loc[mask_attrition, "Gender"].value_counts(normalize=True)* 100).reset_index().round(2)


,Gender,proportion
0,Male,63.29
1,Female,36.71


In [13]:
# Con la consulta filtrada, podemos ver que del total de renuncias, el 63.29% corresponde a hombres mientras que el 36.71% son mujeres.

In [14]:
# 🎯Pregunta:“¿Influye el género en la conciliación (WorkLifeBalance) y en la rotación (Attrition)?”

attrition_gender_wlb = (df.groupby(["Gender", "WorkLifeBalance"])["Attrition"].mean()* 100).reset_index().round(2).sort_values("Attrition", ascending = False)
display(attrition_gender_wlb)

,Gender,WorkLifeBalance,Attrition
4,Male,1,36.00
3,Female,4,24.56
0,Female,1,23.33
5,Male,2,18.75
6,Male,3,15.15
1,Female,2,13.97
7,Male,4,13.54
2,Female,3,12.88


In [15]:
df.groupby(["Gender","WorkLifeBalance"]).size()

Gender  WorkLifeBalance
Female  1                   30
        2                  136
        3                  365
        4                   57
Male    1                   50
        2                  208
        3                  528
        4                   96
dtype: int64

In [16]:
wlb_gender = (
    df.groupby(["Gender", "WorkLifeBalance"])
      .agg(
          AttritionRate=("Attrition", "mean"),
          Count=("Attrition", "count")
      )
      .reset_index()
)

wlb_gender["AttritionRate"] *= 100
wlb_gender.round(1)


,Gender,WorkLifeBalance,AttritionRate,Count
0,Female,1,23.3,30
1,Female,2,14.0,136
2,Female,3,12.9,365
3,Female,4,24.6,57
4,Male,1,36.0,50
5,Male,2,18.8,208
6,Male,3,15.2,528
7,Male,4,13.5,96


In [17]:
women = df[df["Gender"] == "Female"]

wlb_joblevel = (
    women.groupby(["WorkLifeBalance", "JobLevel"])
         .agg(
             AttritionRate=("Attrition", "mean"),
             Count=("Attrition", "count")
         )
         .reset_index()
)

wlb_joblevel["AttritionRate"] *= 100


In [18]:
wlb_tenure = (women.groupby(["WorkLifeBalance", "TenureGroup"]).agg(AttritionRate=("Attrition", "mean"), Count=("Attrition", "count")).reset_index())

wlb_tenure["AttritionRate"] *= 100


#### 💡Insight: 
Los empleados con peor conciliación (WorkLifeBalance = 1) presentan tasas de rotación mucho más elevadas, especialmente en hombres. Esto sugiere que políticas de flexibilidad podrían mejorar significativamente la retención.

### Attrition por departmento

En esta sección se analiza qué departamentos presentan mayor proporción de rotación.

El objetivo es identificar áreas internas donde la retención es más crítica.

In [19]:
df['Department'].value_counts().reset_index()

,Department,count
0,Research & Development,968
1,Sales,439
2,Human Resources,63


In [20]:
# 🎯 Pregunta: ¿en qué departamentos se va más la gente?
# En este caso, vamos a agrupar teniendo en cuenta 'Department'.
# Recordemos que 'Attrition' es una columna binaria, por lo que al calcular la media solamente se calcularía la correspondiente al valor 1 (se van)

attrition_by_dept = ((df.groupby("Department")["Attrition"].mean())*100).sort_values(ascending=False).reset_index()
display(attrition_by_dept.round(2))

,Department,Attrition
0,Sales,20.27
1,Human Resources,19.05
2,Research & Development,14.05


#### 💡Insight: 
El departamento de Sales muestra una tasa de rotación superior al resto, lo que sugiere que puede tratarse de un entorno más exigente o con menor estabilidad laboral.

### Attrition por puesto de trabajo

Se analiza qué puestos presentan mayor proporción de abandono.

Este análisis permite detectar roles especialmente vulnerables dentro de la organización.

In [21]:
# 🎯 Pregunta: ¿qué puestos son los que más abandona la gente?
# Vamos a analizar en qué puestos ('JobRole') hay un mayor porcentaje de rotación.

attrition_by_jobrole = ((df.groupby("JobRole")["Attrition"].mean())*100).sort_values(ascending=False).reset_index()
display(attrition_by_jobrole.round(2))

,JobRole,Attrition
0,Sales Representative,39.76
1,Laboratory Technician,23.94
2,Human Resources,23.08
3,Sales Executive,17.48
4,Research Scientist,16.10
5,Manufacturing Director,6.90
6,Healthcare Representative,6.87
7,Manager,4.90
8,Research Director,2.50


#### 💡 Insight: 
Roles como Sales Representative y Laboratory Technician presentan los mayores niveles de rotación, lo que indica que podrían requerir medidas específicas de motivación, desarrollo o condiciones laborales.

### Job Level y Attrition

Se analiza si el nivel jerárquico influye en la rotación dentro de la empresa.


In [22]:
# 🎯 Pregunta: ¿en qué nivel laboral hay más rotación?
# Vamos a analizar en qué niveles ('JobLevel') hay un mayor porcentaje de rotación.

attrition_by_joblevel = ((df.groupby("JobLevel")["Attrition"].mean())*100).sort_values(ascending=False).reset_index()
display(attrition_by_joblevel.round(2))

,JobLevel,Attrition
0,1,26.34
1,3,14.68
2,2,9.74
3,5,7.25
4,4,4.72


#### 💡 Insight: 
La rotación es más elevada en niveles bajos (JobLevel 1–2), mientras que los puestos senior presentan mayor estabilidad. Esto puede deberse a que los empleados junior buscan con más frecuencia oportunidades externas de crecimiento profesional.

### Distancia y Attrition

Se evalúa si la distancia entre el hogar y el lugar de trabajo influye en la rotación, especialmente en ciertos puestos.

La hipótesis es que una mayor distancia puede aumentar el desgaste y la probabilidad de abandono.

In [23]:
# 🎯 Pregunta: ¿influye la distancia al puesto de trabajo con la rotación?
attrition_by_distance = ((df.groupby("DistanceFromHome")["Attrition"].mean())*100).round(2).sort_values(ascending=False).reset_index()
display(attrition_by_distance)

,DistanceFromHome,Attrition
0,24,42.86
1,22,31.58
2,13,31.58
3,12,30.00
4,17,25.00
5,27,25.00
6,25,24.00
7,16,21.88
8,9,21.18
9,15,19.23


In [24]:
# 🎯 Pregunta: ¿influye la distancia  por puesto de trabajo  con la rotación?
JobRole_distance = (df.groupby(["JobRole", "Attrition"])["DistanceFromHome"].mean().round(2).reset_index().sort_values("Attrition", ascending = False))
display(JobRole_distance)

,JobRole,Attrition,DistanceFromHome
1,Healthcare Representative,1,17.67
3,Human Resources,1,13.42
15,Sales Executive,1,12.65
5,Laboratory Technician,1,9.66
9,Manufacturing Director,1,8.80
7,Manager,1,10.00
13,Research Scientist,1,9.77
17,Sales Representative,1,8.15
11,Research Director,1,7.00
0,Healthcare Representative,0,9.20


In [25]:
pivot = df.pivot_table(
    values="DistanceFromHome",
    index="JobRole",
    columns="Attrition",
    aggfunc="mean"
)

# Diferencia entre los que se van (1) y los que se quedan (0)
pivot["difference"] = pivot[1] - pivot[0]

# Ordenar por mayor impacto de distancia
pivot = pivot.sort_values("difference", ascending=False)

# Mostrar resultado final
pivot = pivot.rename(columns={0: "Stayed", 1: "Left"})
display(pivot.round(2))

Attrition,Stayed,Left,difference
JobRole,,,
Healthcare Representative,9.20,17.67,8.46
Human Resources,6.60,13.42,6.82
Sales Executive,9.03,12.65,3.62
Manager,7.93,10.00,2.07
Research Scientist,8.87,9.77,0.90
Laboratory Technician,9.33,9.66,0.33
Manufacturing Director,9.53,8.80,-0.73
Sales Representative,9.00,8.15,-0.85
Research Director,8.47,7.00,-1.47


 #### 💡Insight: 
 En puestos como Healthcare Representative, los empleados que abandonan viven considerablemente más lejos que aquellos que permanecen, lo que sugiere que la distancia es un factor relevante en ciertos roles.

In [26]:
# 🎯 Pregunta: ¿influye que el trabajador tenga que desplazarse por motivos de trabajo en la rotación?
attrition_by_travel = ((df.groupby("BusinessTravel")["Attrition"].mean())*100).round(2).sort_values(ascending = False).reset_index()
display(attrition_by_travel)

,BusinessTravel,Attrition
0,Frequently,24.71
1,Rarely,14.98
2,Non-Travel,8.63


In [27]:
attrition_by_travel_role= ((df.groupby(["BusinessTravel", "JobRole"])["Attrition"].mean())*100).round(2).sort_values(ascending = False).reset_index()
display(attrition_by_travel_role)

,BusinessTravel,JobRole,Attrition
0,Frequently,Sales Representative,65.22
1,Frequently,Human Resources,44.44
2,Rarely,Sales Representative,32.14
3,Frequently,Laboratory Technician,31.91
4,Frequently,Research Scientist,26.92
5,Rarely,Laboratory Technician,24.32
6,Frequently,Sales Executive,22.81
7,Rarely,Human Resources,20.51
8,Rarely,Sales Executive,17.24
9,Non-Travel,Research Scientist,14.81


 #### 💡Insight: 
  Aquí tambíen vemos que influye, aquellos puestos que viajan con más frecuencia tienen más taasa de rotación. Lo vemos principalmente en puestos  como Sales Representative.

### Horas extra y Attrition

Se analiza si trabajar horas extra está asociado a una mayor probabilidad de rotación.



In [28]:
# 🎯 Pregunta: ¿la realización de horas extras está relacionada con el abandono de la empresa?
# 'OverTime', al igual que 'Attrition' es una columna binaria (1 = se van/0 = se quedan)

attrition_overtime = (df.groupby("OverTime")["Attrition"].mean()* 100).reset_index()
display(attrition_overtime.round(2))

,OverTime,Attrition
0,0,10.52
1,1,30.86


In [29]:
# Cruzamos dos variables relevantes : 'JobRole' y 'OverTime':
job_overtime = (df.groupby("JobRole")["OverTime"].mean()* 100).round(2).sort_values(ascending = False).reset_index()
display(job_overtime)

,JobRole,OverTime
0,Research Scientist,32.53
1,Sales Representative,28.92
2,Healthcare Representative,28.24
3,Sales Executive,28.22
4,Research Director,27.50
5,Manager,26.47
6,Manufacturing Director,25.52
7,Human Resources,23.08
8,Laboratory Technician,22.78


In [30]:
# Veamos el nivel del puesto con la realización de horas extras:
job_level_overtime = (df.groupby("JobLevel")["OverTime"].mean()* 100).round(2).sort_values(ascending = False).reset_index()
display(job_level_overtime)

,JobLevel,OverTime
0,4,31.13
1,1,27.62
2,3,27.52
3,2,26.97
4,5,26.09


#### 💡 Insight: 
Los empleados que realizan horas extra presentan una tasa de abandono significativamente superior. Esto sugiere que la carga de trabajo y el desgaste laboral pueden ser factores determinantes en la decisión de salida.

Vamos a relacionar los desplazamientos por trabajo, las horas extras y los distintos puestos, y si todas estas variables pueden relacionarse con tasas de rotación más altas.

In [31]:
# 🎯 Pregunta: ¿Qué porcentaje de empleados hace horas extra según su nivel de viajes?

travel_overtime = (df.groupby("BusinessTravel")["OverTime"].mean().mul(100).round(2)).sort_values(ascending=False).reset_index()
travel_overtime


,BusinessTravel,OverTime
0,Frequently,30.80
1,Rarely,27.15
2,Non-Travel,24.46


In [32]:
# 🎯 Pregunta: ¿Viajar y hacer horas extra multiplica la rotación?

travel_overtime_attrition = (
    df.groupby(["BusinessTravel", "OverTime"])
      .agg(
          AttritionRate=("Attrition", "mean"),
          EmployeeCount=("Attrition", "count"),
          AttritionCount=("Attrition", "sum")   # 👈 bajas reales
      )
      .reset_index()
)

travel_overtime_attrition["AttritionRate"] = (travel_overtime_attrition["AttritionRate"] * 100).round(2)
travel_overtime_attrition = travel_overtime_attrition[['BusinessTravel', 'OverTime', 'EmployeeCount', 'AttritionRate','AttritionCount']]
travel_overtime_attrition = travel_overtime_attrition.sort_values("AttritionRate", ascending=False)
travel_overtime_attrition


,BusinessTravel,OverTime,EmployeeCount,AttritionRate,AttritionCount
1,Frequently,1,81,43.21,35
5,Rarely,1,290,28.62,83
3,Non-Travel,1,34,20.59,7
0,Frequently,0,182,16.48,30
4,Rarely,0,778,9.90,77
2,Non-Travel,0,105,4.76,5


#### 💡 Insight:
Aunque no se puede inferir causalidad, la interacción entre viajes y horas extra se asocia a tasas de rotación significativamente más altas.
Además del porcentaje de rotación, incluimos el número absoluto de bajas por combinación para evaluar el impacto real de cada segmento.

## 2️⃣ Satisfacción y rotación

### Job Satisfaction y Attrition

Se analiza la satisfacción laboral media comparando empleados que permanecen frente a aquellos que abandonan.
La satisfacción es uno de los indicadores más relacionados con la retención.

In [33]:
# 🎯 Pregunta: ¿a menor satisfacción, mayor rotación?
# Tomaremos la columna 'JobSatisfaction' para relacionarla con la tasa de rotación. Esta primera columna alberga registros
# con valores posibles de 1 a 4, obtenidos mediante encuenstas y siendo 1 = NADA SATISFECHO y 4 = MUY SATISFECHO.

attrition_job_sat = (df.groupby("JobSatisfaction")["Attrition"].mean()* 100).round(2).reset_index()
display(attrition_job_sat)

,JobSatisfaction,Attrition
0,1,22.61
1,2,16.73
2,3,16.63
3,4,11.21


In [34]:
job_sat_counts = (df["JobSatisfaction"].value_counts().reset_index().rename(columns={"count" : "Employees"}))
display(job_sat_counts)

,JobSatisfaction,Employees
0,3,457
1,4,455
2,1,283
3,2,275


In [35]:
# Vamos a unir los dos dataframe:
job_sat_analysis = job_sat_counts.merge( attrition_job_sat, on="JobSatisfaction", how="left").sort_values(by='JobSatisfaction')
display(job_sat_analysis)


,JobSatisfaction,Employees,Attrition
2,1,283,22.61
3,2,275,16.73
0,3,457,16.63
1,4,455,11.21


In [36]:
#Añadimos una nueva columna con el número de empleados que han dejado la empresa.
job_sat_analysis["Leavers"] = (job_sat_analysis["Employees"] * job_sat_analysis["Attrition"] / 100).round().astype(int)
job_sat_analysis = job_sat_analysis[['JobSatisfaction','Employees', 'Attrition', 'Leavers']]
display(job_sat_analysis)

,JobSatisfaction,Employees,Attrition,Leavers
2,1,283,22.61,64
3,2,275,16.73,46
0,3,457,16.63,76
1,4,455,11.21,51


### 💡 Insight: 
 Los empleados que abandonan muestran niveles más bajos de satisfacción laboral.

### Environment Satisfaction y Attrition

Se analiza si el entorno laboral percibido afecta a la rotación.



In [37]:
# 🎯 Pregunta: ¿el ambiente laboral se ve reflejado en la rotación?
# Para resolver esta pregunta, recurrimos a la columna 'EnvironmentSatisfaction'. Al igual que ocurre con 'JobSatisfaction', los valores
# posibles en 'EnvironmentSatisfaction' van de 1 a 4, siendo 1 = NADA SATISFECHO y 4 = MUY SATISFECHO.

attrition_env_sat = (df.groupby("EnvironmentSatisfaction")["Attrition"].mean()* 100).round(2).reset_index()
display(attrition_env_sat)

,EnvironmentSatisfaction,Attrition
0,1,25.35
1,2,14.98
2,3,13.69
3,4,13.45


In [38]:
env_counts = (df["EnvironmentSatisfaction"].value_counts().reset_index().rename(columns={
          "EnvironmentSatisfaction": "EnvironmentSatisfaction",
          "count" : "Employees"
      })
)
display(env_counts)

,EnvironmentSatisfaction,Employees
0,3,453
1,4,446
2,2,287
3,1,284


In [39]:
# Vamos a unir los dos dataframe:
env_analysis = env_counts.merge( attrition_env_sat, on="EnvironmentSatisfaction", how="left").sort_values(by='EnvironmentSatisfaction')
display(env_analysis)


,EnvironmentSatisfaction,Employees,Attrition
3,1,284,25.35
2,2,287,14.98
0,3,453,13.69
1,4,446,13.45


In [40]:
#Añadimos una nueva columna con el número de empleados que han dejado la empresa.
env_analysis["Leavers"] = (env_analysis["Employees"] * env_analysis["Attrition"] / 100).round().astype(int)
display(env_analysis)

,EnvironmentSatisfaction,Employees,Attrition,Leavers
3,1,284,25.35,72
2,2,287,14.98,43
0,3,453,13.69,62
1,4,446,13.45,60


### 💡 Insight: 
Una menor satisfacción con el ambiente de trabajo está asociada a una mayor rotación. Esto indica que factores como cultura, clima laboral o condiciones físicas pueden influir en la retención.

### Work-Life Balance y Attrition

Se evalúa si la conciliación vida-trabajo influye en la retención.


In [41]:
# 🎯 Pregunta: ¿la conciliación laboral afecta a la rotación?
# Para resolver esta pregunta, recurrimos a la columna 'WorkLifeBalance'. Al igual que ocurre con 'JobSatisfaction', los valores
# posibles en 'EnvironmentSatisfaction' van de 1 a 4, siendo 1 = NADA SATISFECHO y 4 = MUY SATISFECHO.

attrition_work_bal = (df.groupby("WorkLifeBalance")["Attrition"].mean()* 100).round(2).reset_index()
display(attrition_work_bal)

,WorkLifeBalance,Attrition
0,1,31.25
1,2,16.86
2,3,14.22
3,4,17.65


In [42]:
work_bal_counts = (df["WorkLifeBalance"].value_counts().reset_index().rename(columns={"count" : "Employees"}))
display(work_bal_counts)

,WorkLifeBalance,Employees
0,3,893
1,2,344
2,4,153
3,1,80


In [43]:
# Vamos a unir los dos dataframe:
work_bal_analysis = work_bal_counts.merge( attrition_work_bal, on="WorkLifeBalance", how="left").sort_values(by='WorkLifeBalance')
display(work_bal_analysis)


,WorkLifeBalance,Employees,Attrition
3,1,80,31.25
1,2,344,16.86
0,3,893,14.22
2,4,153,17.65


In [44]:
#Añadimos una nueva columna con el número de empleados que han dejado la empresa.
work_bal_analysis["Leavers"] = (work_bal_analysis["Employees"] * work_bal_analysis["Attrition"] / 100).round().astype(int)
work_bal_analysis = work_bal_analysis[['WorkLifeBalance','Employees', 'Attrition', 'Leavers']]
display(work_bal_analysis)

,WorkLifeBalance,Employees,Attrition,Leavers
3,1,80,31.25,25
1,2,344,16.86,58
0,3,893,14.22,127
2,4,153,17.65,27


#### 💡 Insight: 
Los empleados con niveles bajos de WorkLifeBalance presentan tasas de rotación notablemente superiores. Esto sugiere que iniciativas de flexibilidad laboral podrían reducir el attrition de forma significativa.

### Relationship Satisfaction y Attrition

Se explora si las relaciones interpersonales en el entorno laboral influyen en la salida de empleados.

In [45]:
# 🎯 Pregunta: ¿cómo afecta la relación entre compañeros al porcentaje de abandono de la empresa?
# Para resolver esta pregunta, recurrimos a la columna 'RelationshipSatisfaction'. Al igual que ocurre con 'JobSatisfaction', los valores
# posibles en 'EnvironmentSatisfaction' van de 1 a 4, siendo 1 = NADA SATISFECHO y 4 = MUY SATISFECHO.

attrition_rel_sat = (df.groupby("RelationshipSatisfaction")["Attrition"].mean()* 100).round(2).reset_index()
display(attrition_rel_sat)

,RelationshipSatisfaction,Attrition
0,1,20.65
1,2,14.85
2,3,15.47
3,4,14.81


In [46]:
rel_sat_counts = (df["RelationshipSatisfaction"].value_counts().reset_index().rename(columns={"count" : "Employees"}))
display(rel_sat_counts)

,RelationshipSatisfaction,Employees
0,3,459
1,4,432
2,2,303
3,1,276


In [47]:
# Vamos a unir los dos dataframe:
rel_sat_analysis = rel_sat_counts.merge( attrition_rel_sat, on="RelationshipSatisfaction", how="left").sort_values(by='RelationshipSatisfaction')
display(rel_sat_analysis)


,RelationshipSatisfaction,Employees,Attrition
3,1,276,20.65
2,2,303,14.85
0,3,459,15.47
1,4,432,14.81


In [48]:
#Añadimos una nueva columna con el número de empleados que han dejado la empresa.
rel_sat_analysis["Leavers"] = (rel_sat_analysis["Employees"] * rel_sat_analysis["Attrition"] / 100).round().astype(int)
rel_sat_analysis = rel_sat_analysis[['RelationshipSatisfaction','Employees', 'Attrition', 'Leavers']]
display(rel_sat_analysis)

,RelationshipSatisfaction,Employees,Attrition,Leavers
3,1,276,20.65,57
2,2,303,14.85,45
0,3,459,15.47,71
1,4,432,14.81,64


#### 💡 Insight: 
Los empleados con menor satisfacción en sus relaciones laborales tienden a abandonar con más frecuencia, lo que refuerza la importancia del liderazgo, la cohesión de equipo y la comunicación interna.

## 3️⃣ Variables clave

### Salario mensual y Attrition

Se evalúa la relación entre el nivel de ingresos y la rotación de empleados.


In [49]:
# 🎯 Pregunta: ¿El salario relativo importa?
# Analizamos la variable que hemos creado 'IncomeBand' con la rotación de empleados.

attrition_income = (df.groupby("IncomeBand")["Attrition"].mean()* 100).sort_values(ascending = False).reset_index()
display(attrition_income.round(2))


,IncomeBand,Attrition
0,Low,51.72
1,Medium,17.07
2,High,5.69


In [50]:
department_income = df.groupby("Department")["MonthlyIncome"].agg(['min', 'max', 'mean','median']).round(0).reset_index()
department_income

,Department,min,max,mean,median
0,Human Resources,1555.0,19717.0,6573.0,3886.0
1,Research & Development,1009.0,19999.0,6269.0,4412.0
2,Sales,1052.0,19847.0,6945.0,5744.0


In [51]:
role_income = df.groupby("JobRole")["MonthlyIncome"].agg(['min', 'max']).round(0).reset_index()
role_income = role_income.sort_values(by="min")
role_income

,JobRole,min,max
6,Research Scientist,1009.0,9724.0
8,Sales Representative,1052.0,6632.0
2,Laboratory Technician,1102.0,7403.0
1,Human Resources,1555.0,10725.0
0,Healthcare Representative,4000.0,13966.0
7,Sales Executive,4001.0,13872.0
4,Manufacturing Director,4011.0,13973.0
3,Manager,4907.0,19999.0
5,Research Director,11031.0,19973.0


In [52]:
# 🎯 Pregunta: ¿Qué puestos reciben, en promedio, mayores subidas salariales y cuáles menores?
dept_salary_hike = df.groupby("JobRole")["PercentSalaryHike"].agg(['mean', 'median']).round(1).reset_index()
dept_salary_hike = dept_salary_hike.sort_values(by="mean")
dept_salary_hike

,JobRole,mean,median
1,Human Resources,14.8,14.0
7,Sales Executive,14.9,14.0
5,Research Director,15.0,14.0
2,Laboratory Technician,15.0,14.0
3,Manager,15.1,14.0
6,Research Scientist,15.4,14.0
0,Healthcare Representative,15.5,14.0
4,Manufacturing Director,15.6,15.0
8,Sales Representative,15.7,15.0


#### 💡  Insight: 
Los empleados con menor ingreso mensual tienden a abandonar con mayor frecuencia. Esto indica que la compensación económica podría ser un factor relevante en los niveles de retención, especialmente en roles operativos o junior. Vemos tambíen que aunque la media salarial sea parecida entre puestos de trabajos, los mínimos y máximos son bastante dispares, siendo Laboratory Technician y Sales Representative los puestos con salarios mínimos y máximos más bajos. 

### Años desde la última promoción

Se analiza el impacto del tiempo transcurrido desde la última promoción sobre la rotación.

La falta de crecimiento profesional puede ser un driver importante de abandono.

In [53]:
# 🎯 Pregunta: ¿A mayor tiempo sin ascenso, mayor probabilidad de abandono?
attrition_promotion =  (df.groupby("YearsSinceLastPromotion")["Attrition"].mean()* 100).reset_index().sort_values(by ='Attrition', ascending = False)
attrition_promotion = attrition_promotion.round(2)
attrition_promotion


,YearsSinceLastPromotion,Attrition
9,9,23.53
15,15,23.08
7,7,21.05
13,13,20.00
0,0,18.93
6,6,18.75
3,3,17.31
2,2,16.98
10,10,16.67
1,1,13.73


In [54]:
# 🎯 Pregunta:  Media de ascenso por departamento:
years_since_promo = df.groupby("Department")["YearsSinceLastPromotion"].mean().round(2).sort_values(ascending=False).reset_index()
display(years_since_promo)

,Department,YearsSinceLastPromotion
0,Sales,2.33
1,Research & Development,2.15
2,Human Resources,1.78


In [55]:
# Promedio de años desde la última promoción por puesto y por Attrition:
years_since_promo = df.groupby("JobRole")["YearsSinceLastPromotion"].mean().round(2).sort_values(ascending=False).reset_index()
display(years_since_promo)

,JobRole,YearsSinceLastPromotion
0,Manager,4.83
1,Research Director,3.19
2,Healthcare Representative,2.97
3,Sales Executive,2.48
4,Manufacturing Director,2.12
5,Research Scientist,1.51
6,Laboratory Technician,1.42
7,Human Resources,1.27
8,Sales Representative,1.06


In [56]:
#🎯 Pregunta:¿Influyen los años sin ascenso en la tasa de rotación por puesto de trabajo?
# Promedio de años sin promoción por puesto y por Attrition.

# Promedio para los que se quedaron (Attrition = 0)
stayed = df[df["Attrition"] == 0].groupby("JobRole")["YearsSinceLastPromotion"].mean().reset_index()
stayed = stayed.rename(columns={"YearsSinceLastPromotion": "Stayed"})
# Promedio para los que se fueron (Attrition = 1)
left = df[df["Attrition"] == 1].groupby("JobRole")["YearsSinceLastPromotion"].mean().reset_index()
left = left.rename(columns={"YearsSinceLastPromotion": "Left"})

#Union del dataFrame
promotion_df = stayed.merge(left, on="JobRole", how="outer")

#claculamos la difrencia de los que se fueroin vs quedaron.
promotion_df["Difference"] = promotion_df["Left"] - promotion_df["Stayed"]
promotion_df = promotion_df.sort_values("Difference", ascending=False).reset_index(drop=True)

display(promotion_df.round(2))

,JobRole,Stayed,Left,Difference
0,Research Director,2.91,14.00,11.09
1,Healthcare Representative,2.89,4.11,1.23
2,Sales Executive,2.36,3.07,0.71
3,Research Scientist,1.44,1.85,0.41
4,Manager,4.84,4.80,-0.04
5,Manufacturing Director,2.15,1.70,-0.45
6,Laboratory Technician,1.55,1.02,-0.53
7,Human Resources,1.40,0.83,-0.57
8,Sales Representative,1.36,0.61,-0.75


#### 💡 Insight:
El puesto de Research Director destaca significativamente: los empleados que abandonan llevan más de 11 años sin promoción en promedio, lo que sugiere que la ausencia de progresión profesional es un factor crítico de rotación en roles senior.


### Satisfacción por puesto de trabajo
Se analiza la satisfacción laboral media por puesto de trabajo comparando empleados que permanecen frente a aquellos que abandonan.

La satisfacción es uno de los indicadores más relacionados con la retención.

In [57]:
#🎯 Pregunta:¿Influyen los años sin ascenso en la satisfacción("JobSatisfaction") por puesto de trabajo y en la rotación (Attrition)?

# Promedio de JobSatisfaction para los que se quedaron
stayed = df[df["Attrition"] == 0].groupby("JobRole")["JobSatisfaction"].mean().reset_index()
stayed = stayed.rename(columns={"JobSatisfaction": "Stayed"})

# Promedio de JobSatisfaction para los que se fueron
left = df[df["Attrition"] == 1].groupby("JobRole")["JobSatisfaction"].mean().reset_index()
left = left.rename(columns={"JobSatisfaction": "Left"})

# Crear nuevo DataFrame combinando ambos
satisfaction_df = stayed.merge(left, on="JobRole", how="outer")

# Columna de diferencia
satisfaction_df["Difference"] = satisfaction_df["Left"] - satisfaction_df["Stayed"]

# Ordenar por la diferencia
satisfaction_df = satisfaction_df.sort_values("Difference", ascending=False).reset_index(drop=True)

# Mostrar resultado
display(satisfaction_df.round(2))


,JobRole,Stayed,Left,Difference
0,Manager,2.72,2.80,0.08
1,Healthcare Representative,2.80,2.78,-0.03
2,Manufacturing Director,2.70,2.60,-0.10
3,Research Director,2.73,2.50,-0.23
4,Sales Executive,2.81,2.51,-0.30
5,Human Resources,2.68,2.33,-0.34
6,Laboratory Technician,2.79,2.44,-0.35
7,Sales Representative,2.90,2.48,-0.42
8,Research Scientist,2.84,2.43,-0.42


 #### 💡Insight: 
En la mayoría de roles, los empleados que abandonan muestran niveles más bajos de satisfacción laboral, especialmente en Sales Representative y Research Scientist. Esto refuerza la relación directa entre satisfacción y rotación.

In [58]:
monthly_income = df.groupby(["Gender", "TenureGroup", "JobLevel"])["MonthlyIncome"].agg(['min', 'max']).reset_index().sort_values(by = ['JobLevel','Gender'])
display(monthly_income.round(0))

,Gender,TenureGroup,JobLevel,min,max
0,Female,0-2,1,1129.0,4907.0
5,Female,10+,1,2011.0,3919.0
10,Female,3-5,1,2001.0,4963.0
15,Female,6-9,1,2153.0,4968.0
20,Male,0-2,1,1009.0,4907.0
25,Male,10+,1,2028.0,4723.0
30,Male,3-5,1,2013.0,4936.0
35,Male,6-9,1,2058.0,4876.0
1,Female,0-2,2,2372.0,9854.0
6,Female,10+,2,2133.0,8381.0


In [59]:
# 🎯 Pregunta: ¿Existen patrones generacionales en la rotación?
# Tomamos otra de las variables que hemos creado, 'AgeGroup', para relacionarla con la tasa de rotación,

attrition_age = (df.groupby("AgeGroup")["Attrition"].mean()* 100).sort_values(ascending = False).reset_index()
display(attrition_age.round(2))


,AgeGroup,Attrition
0,Under 25,38.89
1,25-45,15.24
2,Over 45,12.02


In [ ]:
#agrupar por rango de edad y contar cuantos empleados hay por grupo de edad.
age_counts = (df["AgeGroup"].value_counts().reset_index().rename(columns={"count" : "Employees"}))
display(age_counts)

,AgeGroup,Employees
0,25-45,1122
1,Over 45,258
2,Under 25,90


In [60]:
# 🎯 Pregunta: ¿dónde se produce la “fuga” real?
# Vamos a ver en qué rango de antigüedad existe el mayor abandono.

attrition_tenure = (df.groupby("TenureGroup")["Attrition"].mean()* 100).sort_values(ascending = False).reset_index()
display(attrition_tenure.round(2))


,TenureGroup,Attrition
0,0-2,29.82
1,3-5,13.82
2,6-9,11.28
3,10+,10.38


In [5]:
#agrupar por rango de antiguedad y contar cuantos empleados hay por grupo de antiguedad.
tenure_counts = (df["TenureGroup"].value_counts().reset_index().rename(columns={"count" : "Employees"}))
display(tenure_counts)

,TenureGroup,Employees
0,3-5,434
1,10+,366
2,0-2,342
3,6-9,328


### Formación vs attrition
Se nuestra muestra el número de formaciones que se dieron a lo largo del año, comparado con la tasa de rotación. 

In [61]:
# 🎯 Pregunta: ¿influye la formación en la atsa de attrition?
#  Agrupar y calcular porcentaje de rotación por trainning durante el último año.
attrition_TTL = df.groupby('TrainingTimesLastYear')['Attrition'].mean().reset_index()
attrition_TTL['Attrition'] = attrition_TTL['Attrition'] * 100  # convertir a %
display(attrition_TTL.round(2))

,TrainingTimesLastYear,Attrition
0,0,28.30
1,1,12.31
2,2,18.53
3,3,13.63
4,4,21.37
5,5,11.71
6,6,9.52


 #### 💡Insight: 
 Al analizar la rotación por número de trainings, observamos que los empleados con menos capacitaciones durante el último año presentan un mayor porcentaje de rotación.

Esto indica que ofrecer más oportunidades de formación podría ayudar a mejorar la retención y evitar la rotación laboral.

 ## 4️⃣ Validar nuestro RiskScore

Este bloque agrupa a los empleados por su RiskScore y calcula el porcentaje de rotación real (Attrition) para cada score. Esto nos permite validar que el RiskScore tiene sentido: a mayor puntuación, mayor proporción de empleados que han dejado la empresa.

 #### 💡Insight: 

Se espera que los empleados con RiskScore alto (por ejemplo ≥6) tengan una rotación significativamente mayor que los de score bajo. Esto confirma que el score refleja correctamente el riesgo de abandono.

In [62]:
# Comparamos nuestro RiskScore con el porcentaje de rotación.
attrition_risk = (df.groupby("RiskScore")["Attrition"].mean()* 100).reset_index()
display(attrition_risk.round(2))


,RiskScore,Attrition
0,0,2.33
1,1,9.92
2,2,9.38
3,3,13.45
4,4,16.01
5,5,24.02
6,6,30.43
7,7,46.67
8,8,52.17
9,9,40.00


Aquí calculamos el RiskScore promedio por rol y el número de empleados por cada rol. Esto permite identificar qué roles tienen un riesgo promedio más alto.


In [63]:
role_risk= df.groupby("JobRole").agg(AvgRiskScore=("RiskScore", "mean"), EmployeeCount=("RiskScore", "count")).round(2).sort_values("AvgRiskScore", ascending=False)
role_risk

,AvgRiskScore,EmployeeCount
JobRole,,
Sales Representative,3.54,83
Research Scientist,3.50,292
Laboratory Technician,3.28,259
Healthcare Representative,3.12,131
Research Director,3.11,80
Sales Executive,3.11,326
Manager,3.10,102
Manufacturing Director,3.07,145
Human Resources,2.87,52


#### 💡Insight:

Roles como Sales Executive o Laboratory Technician pueden presentar un AvgRiskScore alto, lo que indica que son perfiles críticos que podrían requerir estrategias de retención específicas.


Se define un umbral de alto riesgo (RiskScore >= 6) y se calcula el porcentaje de empleados de alto riesgo por rol. Esto identifica roles que concentran la mayor parte del riesgo.


In [64]:
high_risk_threshold = 6

high_risk_by_role = (
    df.assign(HighRisk=df["RiskScore"] >= high_risk_threshold)
      .groupby("JobRole")
      .agg(
          HighRiskPercent = ("HighRisk", "mean"),
          EmployeeCount=("RiskScore", "count")
      )
      .sort_values("HighRiskPercent", ascending=False)
)

# pasar a porcentaje
high_risk_by_role["HighRiskPercent"] *= 100

# redondear (presentación)
high_risk_by_role["HighRiskPercent"] = high_risk_by_role["HighRiskPercent"].round(1)

high_risk_by_role


,HighRiskPercent,EmployeeCount
JobRole,,
Research Scientist,17.8,292
Sales Representative,14.5,83
Manufacturing Director,13.1,145
Manager,11.8,102
Laboratory Technician,11.6,259
Healthcare Representative,9.9,131
Sales Executive,9.8,326
Research Director,8.8,80
Human Resources,7.7,52


#### 💡Insight:

Un % alto de los Research Scientists y Sales Reprsentative puede estar en alto riesgo, indicando que este grupo requiere medidas de retención prioritarias.

In [65]:
df.groupby("JobRole")["RiskScore"].agg(['median','max']).reset_index()

,JobRole,median,max
0,Healthcare Representative,3.0,8
1,Human Resources,3.0,7
2,Laboratory Technician,3.0,9
3,Manager,3.0,8
4,Manufacturing Director,3.0,8
5,Research Director,3.0,9
6,Research Scientist,3.0,10
7,Sales Executive,3.0,9
8,Sales Representative,4.0,8


Se agrupa por antigüedad en la empresa (TenureGroup) y se calcula el porcentaje de empleados de alto riesgo. Esto ayuda a identificar si ciertos grupos de antigüedad presentan mayor riesgo.


In [66]:
high_risk_threshold = 6

high_risk_by_tenure = (
    df.assign(HighRisk=df["RiskScore"] >= high_risk_threshold)
      .groupby("TenureGroup")
      .agg(
          HighRiskPercent = ("HighRisk", "mean"),
          EmployeeCount=("RiskScore", "count")
      )
      .sort_values("HighRiskPercent", ascending=False)
)

# pasar a porcentaje
high_risk_by_tenure["HighRiskPercent"] *= 100

# redondear (presentación)
high_risk_by_tenure["HighRiskPercent"] = high_risk_by_tenure["HighRiskPercent"].round(1)

high_risk_by_tenure


,HighRiskPercent,EmployeeCount
TenureGroup,,
0-2,18.7,342
3-5,11.1,434
6-9,11.0,328
10+,9.0,366



#### 💡Insight:

Los empleados con menos de 2 años podrían mostrar un % de alto riesgo, indicando que la retención temprana es un punto crítico para la empresa.

Se agrupa por rango de edad (AgeGroup) y se calcula el porcentaje de empleados de alto riesgo. Esto permite detectar qué rangos de edad concentran mayor riesgo de rotación.

In [67]:
high_risk_threshold = 6

high_risk_by_age = (
    df.assign(HighRisk=df["RiskScore"] >= high_risk_threshold)
      .groupby("AgeGroup")
      .agg(
          HighRiskPercent = ("HighRisk", "mean"),
          EmployeeCount=("RiskScore", "count")
      )
      .sort_values("HighRiskPercent", ascending=False)
)

# pasar a porcentaje
high_risk_by_age["HighRiskPercent"] *= 100

# redondear (presentación)
high_risk_by_age["HighRiskPercent"] = high_risk_by_age["HighRiskPercent"].round(1)

high_risk_by_age


,HighRiskPercent,EmployeeCount
AgeGroup,,
Under 25,17.8,90
Over 45,12.4,258
25-45,11.9,1122



#### 💡Insight:

Los empleados jóvenes (menores de 25) pueden concentrar un porcentaje alto de riesgo, sugiriendo que programas de retención enfocados en este grupo podrían ser efectivos.

---

#  CONCLUSIONES FINALES

A partir del análisis, se identifican varios factores clave asociados a la rotación de empleados:



## DRIVERS PRINCIPALES DE ATTRITION



#### Trabajo y horas extras
Los empleados que realizan horas extra muestran una tasa de abandono significativamente superior. Esto sugiere que la sobrecarga de trabajo y el desgaste pueden ser factores determinantes en la decisión de salida.

---

#### Compensación y salario mensual
Se observa que los empleados con menores ingresos mensuales tienden a abandonar con mayor frecuencia. Esto indica que la compensación económica puede ser un factor crítico, especialmente en roles junior o técnicos. Además hay ciertos puestios que tienen má scarga laboral y responsabilidad lo que no se traduce en dinero.

---

#### Crecimiento laboral y nivel de trabajo
La rotación es más elevada en niveles jerárquicos bajos (JobLevel 1–2), mientras que los empleados senior presentan mayor estabilidad. Esto podría reflejar que los perfiles más jóvenes buscan oportunidades externas de desarrollo profesional.

---

#### Experiencia y tasa de rotación
Los abandonos se concentran principalmente en empleados con pocos años de experiencia total y poca antigüedad en la empresa. Esto refuerza la importancia de fortalecer el onboarding y el acompañamiento durante los primeros años.

---

#### Trabajo y satisfacción en el ambiente de trabajo
Los empleados con menor satisfacción laboral, menor satisfacción con el entorno o relaciones laborales más débiles presentan mayores niveles de rotación. Esto destaca la relevancia del clima organizacional, la cultura interna y la calidad del liderazgo.

---

#### Impacto en los desplazamientos y viajes
Los empleados que viajan con frecuencia muestran mayor propensión al abandono, lo que sugiere que la exigencia asociada a desplazamientos continuos puede afectar negativamente al bienestar y la retención.

---


## RECOMENDACIONES ESTRATÉGICAS

-  Desarrollar planes de carrera claros para empleados junior, fomentando la promoción interna.

- Mejorar programas de engagement en puestos con alta rotación.

- Evaluar políticas de trabajo híbrido para empleados con largas distancias.

- Evaluar alternativas para reducir la carga asociada a viajes frecuentes.

- Desarrollar iniciativas de bienestar y flexibilidad laboral.

- Implementar estrategias para reducir la carga excesiva de trabajo y controlar el impacto de las horas extras.

- Revisar políticas salariales y beneficios para perfiles con ingresos bajos y alta rotación. Replantear las subidas salariales y forma de reparirlo.

- Reforzar el proceso de onboarding y seguimiento en los primeros años, donde ocurre la mayoría de abandonos.

- Mejorar iniciativas de bienestar, conciliación y clima laboral, especialmente en roles críticos.

---


## TIP FINAL
"Prestar especial atención a los puestos de Sales Representative, Laboratory Technician, y Research Scientist en edades más jóvenes, con menos años en la empresa, ya que son los que presentan más carga de trabajo (overtime), tienen que viajar más por cuestiones de trabajo, tienen sueldos más bajos, tienen menos conciliación laboral y están más descontestos. Son puestos  de un nivel bajo(1) pero tienen unas exigencias muy grandes lo que puede infliur en que la tasa de rotación aquí sea mayor. Incentivarlos y ayudarlos será clave en la retención de empleados."
